In [1]:
import numpy as np
import pandas as pd
import sqlalchemy
import pyodbc

In [2]:
# setting up engine connection

engine = sqlalchemy.create_engine("mssql+pyodbc://Trusted_Connection=yes@bikestores")

engine.connect()

print(engine)

Engine(mssql+pyodbc://Trusted_Connection=yes@bikestores)


In [3]:
# extension for sql statements

%load_ext sql

%sql mssql+pyodbc://Trusted_Connection=yes@bikestores


In [4]:
# sql statement to select tables

sales_table = '''SELECT O.order_id, O.order_date, O.customer_id, CU.state, CU.zip_code, OI.quantity, OI.list_price,
	PR.product_id, PR.product_name, BR.brand_id, BR.brand_name, O.staff_id
FROM sales.orders AS O
	INNER JOIN sales.order_items AS OI
	ON O.order_id = OI.order_id
		INNER JOIN sales.customers AS CU
		ON O.customer_id = CU.customer_id
			INNER JOIN production.products AS PR
			ON OI.product_id = PR.product_id
				INNER JOIN production.brands AS BR
				ON PR.brand_id = BR.brand_id;'''

sales_table= %sql $sales_table




 * mssql+pyodbc://Trusted_Connection=yes@bikestores
Done.


In [5]:
# convert  tables to data frame

sales_df = sales_table.DataFrame()

sales_df

,order_id,order_date,customer_id,state,zip_code,quantity,list_price,product_id,product_name,brand_id,brand_name,staff_id
0,1,2016-01-01,259,CA,94566,1,599.99,20,Electra Townie Original 7D EQ - Women's - 2016,1,Electra,2
1,1,2016-01-01,259,CA,94566,2,1799.99,8,Trek Remedy 29 Carbon Frameset - 2016,9,Trek,2
2,1,2016-01-01,259,CA,94566,2,1549.00,10,Surly Straggler - 2016,8,Surly,2
3,1,2016-01-01,259,CA,94566,2,599.99,16,Electra Townie Original 7D EQ - 2016,1,Electra,2
4,1,2016-01-01,259,CA,94566,1,2899.99,4,Trek Fuel EX 8 29 - 2016,9,Trek,2
...,...,...,...,...,...,...,...,...,...,...,...,...
4717,1614,2018-11-28,135,TX,77016,2,2299.99,159,Trek Emonda ALR 6 - 2018,9,Trek,8
4718,1614,2018-11-28,135,TX,77016,2,269.99,213,Electra Cruiser 1 - 2016/2017/2018,1,Electra,8
4719,1615,2018-12-28,136,TX,75126,2,2299.99,197,Trek Verve+ Lowstep - 2018,9,Trek,8
4720,1615,2018-12-28,136,TX,75126,1,899.99,214,Electra Tiger Shark 3i - 2018,1,Electra,8


In [6]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4722 entries, 0 to 4721
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   order_id      4722 non-null   int64 
 1   order_date    4722 non-null   object
 2   customer_id   4722 non-null   int64 
 3   state         4722 non-null   object
 4   zip_code      4722 non-null   object
 5   quantity      4722 non-null   int64 
 6   list_price    4722 non-null   object
 7   product_id    4722 non-null   int64 
 8   product_name  4722 non-null   object
 9   brand_id      4722 non-null   int64 
 10  brand_name    4722 non-null   object
 11  staff_id      4722 non-null   int64 
dtypes: int64(6), object(6)
memory usage: 442.8+ KB


### Augment data with additional columns

In [7]:
# Creating sales column

sales_df['sales'] = sales_df['quantity'] * sales_df['list_price']

sales_df

,order_id,order_date,customer_id,state,zip_code,quantity,list_price,product_id,product_name,brand_id,brand_name,staff_id,sales
0,1,2016-01-01,259,CA,94566,1,599.99,20,Electra Townie Original 7D EQ - Women's - 2016,1,Electra,2,599.99
1,1,2016-01-01,259,CA,94566,2,1799.99,8,Trek Remedy 29 Carbon Frameset - 2016,9,Trek,2,3599.98
2,1,2016-01-01,259,CA,94566,2,1549.00,10,Surly Straggler - 2016,8,Surly,2,3098.00
3,1,2016-01-01,259,CA,94566,2,599.99,16,Electra Townie Original 7D EQ - 2016,1,Electra,2,1199.98
4,1,2016-01-01,259,CA,94566,1,2899.99,4,Trek Fuel EX 8 29 - 2016,9,Trek,2,2899.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4717,1614,2018-11-28,135,TX,77016,2,2299.99,159,Trek Emonda ALR 6 - 2018,9,Trek,8,4599.98
4718,1614,2018-11-28,135,TX,77016,2,269.99,213,Electra Cruiser 1 - 2016/2017/2018,1,Electra,8,539.98
4719,1615,2018-12-28,136,TX,75126,2,2299.99,197,Trek Verve+ Lowstep - 2018,9,Trek,8,4599.98
4720,1615,2018-12-28,136,TX,75126,1,899.99,214,Electra Tiger Shark 3i - 2018,1,Electra,8,899.99


In [8]:
# Redistribute columns
sales_df = sales_df.reindex(columns=['order_id', 'order_date', 'customer_id', 'state', 'zip_code',
                                        'quantity', 'list_price', 'sales', 'product_id', 'product_name', 
                                        'brand_id', 'brand_name', 'staff_id'])
sales_df

,order_id,order_date,customer_id,state,zip_code,quantity,list_price,sales,product_id,product_name,brand_id,brand_name,staff_id
0,1,2016-01-01,259,CA,94566,1,599.99,599.99,20,Electra Townie Original 7D EQ - Women's - 2016,1,Electra,2
1,1,2016-01-01,259,CA,94566,2,1799.99,3599.98,8,Trek Remedy 29 Carbon Frameset - 2016,9,Trek,2
2,1,2016-01-01,259,CA,94566,2,1549.00,3098.00,10,Surly Straggler - 2016,8,Surly,2
3,1,2016-01-01,259,CA,94566,2,599.99,1199.98,16,Electra Townie Original 7D EQ - 2016,1,Electra,2
4,1,2016-01-01,259,CA,94566,1,2899.99,2899.99,4,Trek Fuel EX 8 29 - 2016,9,Trek,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4717,1614,2018-11-28,135,TX,77016,2,2299.99,4599.98,159,Trek Emonda ALR 6 - 2018,9,Trek,8
4718,1614,2018-11-28,135,TX,77016,2,269.99,539.98,213,Electra Cruiser 1 - 2016/2017/2018,1,Electra,8
4719,1615,2018-12-28,136,TX,75126,2,2299.99,4599.98,197,Trek Verve+ Lowstep - 2018,9,Trek,8
4720,1615,2018-12-28,136,TX,75126,1,899.99,899.99,214,Electra Tiger Shark 3i - 2018,1,Electra,8


### What are the annual sales and all sales combined?

In [9]:
# Change sales column data type to integer
sales_df['sales'] = sales_df['sales'].astype(int)

# Change order_date data type to date
sales_df['order_date'] = pd.to_datetime(sales_df['order_date'], format="%Y/%m/%d")

# Create year column
sales_df['year'] = sales_df['order_date'].dt.year

# Get annual sales 
year_sales = sales_df.groupby('year')['sales'].sum().reset_index()
year_sales['total_sales'] = year_sales['sales'].cumsum()

# Drop year column
sales_df.drop('year', axis=1, inplace=True)

year_sales


,year,sales,total_sales
0,2016,2708014,2708014
1,2017,3843541,6551555
2,2018,2023165,8574720


### What are the april sales from each year?

In [10]:
april_month = (sales_df['order_date'].dt.month == 4)

april_df = sales_df.loc[april_month]

april_df['sales'].sum()


1349892